In [1]:
import tensorflow as tf

In [8]:
def createLSTMLayers(n_layers, n_cells, dropout):
    cell_list = []
    
    for layer in range(n_layers):
        cell = tf.contrib.rnn.LayerNormBasicLSTMCell(n_cells, activation=tf.nn.relu, layer_norm=False)
        
        cell_list.append(cell)
        
    return cell_list

In [3]:
tf.reset_default_graph()

In [9]:
encoder_layers = 2
encoder_cells = 5

decoder_layers = 2
decoder_cells = 5

seq_length = 5
n_features = 1

In [7]:
inputs = tf.placeholder(name='input', shape=[None, seq_length, n_features], dtype=tf.float32)

In [13]:
#encoder

with tf.variable_scope('encoder', reuse=False):
    
    encoder_cell_fw = tf.contrib.rnn.MultiRNNCell(createLSTMLayers(encoder_layers, encoder_cells, 1))
    encoder_cell_bw = tf.contrib.rnn.MultiRNNCell(createLSTMLayers(encoder_layers, encoder_cells, 1))
    
    (encoder_fw_outputs, encoder_bw_outputs), encoder_state_outputs = tf.nn.bidirectional_dynamic_rnn(
                                        encoder_cell_fw,
                                        encoder_cell_bw, 
                                        inputs=inputs,
                                        dtype=tf.float32, time_major=False, swap_memory=True)
    
    encoder_final_state = tf.concat([encoder_state_outputs[0].h for state_tuple in encoder_state_outputs], axis=1)

AttributeError: 'tuple' object has no attribute 'h'

In [15]:
encoder_state_outputs[0]

(LSTMStateTuple(c=<tf.Tensor 'encoder_3/bidirectional_rnn/fw/fw/while/Exit_2:0' shape=(?, 5) dtype=float32>, h=<tf.Tensor 'encoder_3/bidirectional_rnn/fw/fw/while/Exit_3:0' shape=(?, 5) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'encoder_3/bidirectional_rnn/fw/fw/while/Exit_4:0' shape=(?, 5) dtype=float32>, h=<tf.Tensor 'encoder_3/bidirectional_rnn/fw/fw/while/Exit_5:0' shape=(?, 5) dtype=float32>))